In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import langsmith

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
langsmith(project_name=os.getenv("LANGCHAIN_PROJECT"))

### FewShotPromptTemplate

- 답변의 예시를 보여줘서 **AI 가 답변의 예시를 따라서 답변**할 수 있도록 하는 방법입니다.
- `OneShot` 은 예시가 하나일때 사용입니다.
- `FewShot` 은 과거의 이력을 모아서 여러개의 예시를 사용하는 것입니다.(사람이 하는 논리적인 추론과정을 먼저 보여주고 AI가 답변하게끔 유도하는 방법입니다.)
- `ZeroShot` 은 아무런 예시를 보여주지 않는 형태를 말합니다.

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

question = "비트코인의 단위는 무엇인가요?"

print(llm.invoke(question).content)

비트코인의 기본 단위는 '비트코인'이며, 기호로는 BTC라고 표기합니다. 비트코인은 더 작은 단위로도 나눌 수 있는데, 주요 단위로는 '밀리비트코인'(mBTC), '마이크로비트코인'(μBTC, 또는 비트), 그리고 '사토시'가 있습니다. 사토시는 비트코인의 가장 작은 단위로, 1비트코인은 1억 사토시(0.00000001 BTC)와 같습니다. 이러한 단위들은 비트코인의 거래 시 소액 거래를 용이하게 하기 위해 사용됩니다.


In [2]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

examples = [
    {
        "question": "비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    """,
    },
    {
        "question": "리플과 카르다노의 창립자가 같은 대륙 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    """,
    },
    {
        "question": "폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    """,
    },
    {
        "question": "체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    """,
    },
]

#### OneShot

In [6]:
example_prompt = PromptTemplate.from_template(
    "Question:\n {question}\nAnswer:\n {answer}"
)

# 하나만 찾아서 예시를 주었기 때문에 OneShot 입니다.
print(
    example_prompt.format(
        **examples[0]  # ** 문법은 언패킹 문법입니다. {} 를 제거해줍니다.
    )
)

Question:
 비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    


#### FewShow

In [17]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

question = "니어프로토콜은 어떤 체인인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
 비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    

Question:
 리플과 카르다노의 창립자가 같은 대륙 출신인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    

Question:
 폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답

In [18]:
chain = prompt | llm

print(chain.invoke(question).content)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 니어프로토콜은 어떤 기능을 가지고 있나요?  
중간 답변: 니어프로토콜은 분산형 애플리케이션(dApp)을 구축할 수 있는 블록체인 플랫폼입니다.  
추가 질문: 니어프로토콜의 합의 알고리즘은 무엇인가요?  
중간 답변: 니어프로토콜은 나이트샤드(Nightshade)라는 샤딩 기술을 사용하는 합의 알고리즘을 채택하고 있습니다.  
최종 답변은: 니어프로토콜은 분산형 애플리케이션을 구축할 수 있는 블록체인 플랫폼입니다.


### Example Selector

- 예제가 많은 경우 프롬프트에 포함된 예제를 선택할 수 있습니다. 질문과 유사도를 판단해서 전체 예제를 프롬프트에 포함시키지않고 **선택해서** 포함합니다.
- SemanticSimilarityExampleSelector(cosine similarity) : 질문과 유사도 계산을 해서 유사도가 높은 몇 개를 선택해줍니다.
- MaxMarginalRelevanceExampleSelector(MMR 알고리즘) : 유사성은 있으나 다양한 예제를 가져오고 싶을때 사용합니다. 관련성과 유사성과 다양성을 함께 평가합니다.(람다계수로 높으면 관련성을 작을수록 다양성을 중점적으로 예시를 추출합니다.)

프롬프트의 비용을 줄일 수 있는 방법 중에 하나입니다.

In [22]:
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
Chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 예시 목록입니다.
    examples,
    # 유사성을 측정하는 임베딩 클래스입니다.
    OpenAIEmbeddings(),
    # 벡터 데이터베이스 클레스 입니다.
    Chroma,
    # 선택할 예시의 갯수입니다.
    k=1,
)

question = "니어프로토콜은 어떤 체인인가요?"
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시입니다 : \n{question}\n")
for example in selected_examples:
    print(f"question:\n{example['question']}")
    print(f"answer:\n{example['answer']}")

입력에 가장 유사한 예시입니다 : 
니어프로토콜은 어떤 체인인가요?

question:
체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    


In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

examples = [
    {
        "question": "비트코인과 이더리움 중 어느 네트워크가 더 오래되었나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 비트코인은 언제 출시되었나요?
    중간 답변: 비트코인은 2009년에 출시되었습니다.
    추가 질문: 이더리움은 언제 출시되었나요?
    중간 답변: 이더리움은 2015년에 출시되었습니다.
    최종 답변은: 비트코인
    """,
    },
    {
        "question": "리플과 카르다노의 창립자가 같은 대륙 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 리플의 창립자는 누구인가요?
    중간 답변: 리플의 창립자는 크리스 라센입니다.
    추가 질문: 크리스 라센은 어느 대륙 출신인가요?
    중간 답변: 크리스 라센은 북미 출신입니다.
    추가 질문: 카르다노의 창립자는 누구인가요?
    중간 답변: 카르다노의 창립자는 찰스 호스킨슨입니다.
    추가 질문: 찰스 호스킨슨은 어느 대륙 출신인가요?
    중간 답변: 찰스 호스킨슨은 북미 출신입니다.
    최종 답변은: 예
    """,
    },
    {
        "question": "폴리곤과 솔라나 중 거래 속도가 더 빠른 네트워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 폴리곤 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 폴리곤 네트워크는 초당 약 7,000 거래를 처리할 수 있습니다.
    추가 질문: 솔라나 네트워크의 거래 속도는 얼마인가요?
    중간 답변: 솔라나 네트워크는 초당 약 65,000 거래를 처리할 수 있습니다.
    최종 답변은: 솔라나
    """,
    },
    {
        "question": "체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    """,
    },
]

embeddings = OpenAIEmbeddings()
chroma_db = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=chroma_db,
    k=2,
)

example_prompt = PromptTemplate.from_template(
    "Question:\n {question}\nAnswer:\n {answer}"
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

question = "니어프로토콜은 어떤 체인인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    

Question:
 체인링크와 테조스가 같은 스마트 계약 플랫폼인가요?
Answer:
 이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 체인링크는 어떤 플랫폼인가요?
    중간 답변: 체인링크는 스마트 계약을 위한 오라클 네트워크입니다.
    추가 질문: 테조스는 어떤 플랫폼인가요?
    중간 답변: 테조스는 자체 스마트 계약을 지원하는 블록체인 플랫폼입니다.
    최종 답변은: 아니요
    

Question:
 니어프로토콜은 어떤 체인인가요?
Answer:
 


In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)


prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n {question}\nAnswer:\n ",
    input_variables=["question"],
)

chain = prompt | llm | StrOutputParser()

answer = chain.stream(
    {
        "question": "니어프로토콜과 아카시네트워크는 어떤 체인인가요?"
    }  # 질문이 올바르지 않으면 example_selector 처럼 원하는 형식의 답변을 받지 못 할 수도 있습니다.
)

for token in answer:
    print(token, end="", flush=True)

이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 니어프로토콜은 어떤 플랫폼인가요?
    중간 답변: 니어프로토콜은 확장성이 뛰어난 블록체인 플랫폼으로, 스마트 계약과 분산 애플리케이션(dApps)을 지원합니다.
    추가 질문: 아카시네트워크는 어떤 플랫폼인가요?
    중간 답변: 아카시네트워크는 분산형 클라우드 컴퓨팅 플랫폼으로, 블록체인 기술을 사용하여 서버리스 컴퓨팅 환경을 제공합니다.
    최종 답변은: 니어프로토콜과 아카시네트워크는 각각 다른 목적을 가진 체인입니다. 니어프로토콜은 스마트 계약과 dApps 지원을 위한 블록체인 플랫픔이며, 아카시네트워크는 분산형 클라우드 컴퓨팅을 제공하는 플랫폼입니다.

### FewShotChatMessagePromptTemplate

In [3]:
message_examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
    - 회의는 김수진 팀장의 개회사로 시작됨.
    - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
    - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
    - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
    - 박지민은 디지털 마케팅 전략에 대해 발표.
    - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
    - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
    - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
    - 팀원들 간의 아이디어 공유 및 토론.
    - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
    - 다음 회의 날짜 및 시간 확정.
    - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [16]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

embeddings = OpenAIEmbeddings()
chroma_db = Chroma()

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=message_examples,
    embeddings=embeddings,
    vectorstore_cls=chroma_db,
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [12]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n    - 회의는 김수진 팀장의 개회사로 시작됨.\n    - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n    - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n    - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n    - 박지민은 디지털 마케팅 전략에 대해 발표.\n    - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n    - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n    - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n    - 팀원들 간의 아이디어 공유 및 토론.\n    - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n    - 다음 회의 날짜 및 시간 확정.\n    - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [13]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [17]:
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# chain 생성
chain = final_prompt | llm | StrOutputParser()

# 실행 및 결과 출력
answer = chain.stream(question)
for token in answer:
    print(token, end="", flush=True)

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 최현수 프로젝트 매니저가 회의를 개회하고, 목적을 설명함.
   - 목적: 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 보고
   - 황지연 개발자가 개발 진행 상황에 대해 보고.
     - 현재 개발 중인 기능들의 상태와 최근 해결된 문제점들에 대해 설명.
     - 다음 주까지 완료 목표로 설정된 기능들을 공유.
   - 김태영 디자이너가 UI/UX 디자인 진행 상황에 대해 보고.
     - 최근 완성된 디자인 요소들과 사용자 피드백을 바탕으로 수정할 계획들을 공유.
     - 다가오는 주에 집중할 디자인 작업들을 설명.

3. 문제점 및 해결 방안 논의
   - 팀원들이 현재 직면한 기술적, 디자인적 문제점들을 공유.
   - 최현수가 문제 해결을 위한 아이디어와 자원 배분을 조정.

4. 다음 마일스톤 계획
   - 최현수가 다음 주까지의 주요 목표와 마일스톤을 정리하여 발표.
   - 각 팀원의 역할과 기대 결과에 대해 명확히 함.

5. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 황지연 담당.

회의는 모든 안건이 충분히 논의되고 각 팀원의 역할과 목표가 명확히 정해진 후 종료되었다.